In [1]:
import json
import w3lib.html
import random
import os
from datetime import datetime
import requests
import string
import calendar
import html
import locale 
from requests_html import HTMLSession
from requests_html import HTML
import validators
import pandas as pd
from tqdm.notebook import tqdm

import warnings
warnings.filterwarnings('ignore')

In [2]:
URL_SEED = 'https://eltrabajo.cl/portal/author/diario-el-trabajo/page/1/'
XPATH_HREF = "//h3/a/@href"

In [3]:
class Crawler():
    
    USER_AGENT_LIST = [
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/22.0.1207.1 Safari/537.1",
        "Mozilla/5.0 (X11; CrOS i686 2268.111.0) AppleWebKit/536.11 (KHTML, like Gecko) Chrome/20.0.1132.57 Safari/536.11",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.6 (KHTML, like Gecko) Chrome/20.0.1092.0 Safari/536.6",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.6 (KHTML, like Gecko) Chrome/20.0.1090.0 Safari/536.6",
        "Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/19.77.34.5 Safari/537.1",
        "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/536.5 (KHTML, like Gecko) Chrome/19.0.1084.9 Safari/536.5",
        "Mozilla/5.0 (Windows NT 6.0) AppleWebKit/536.5 (KHTML, like Gecko) Chrome/19.0.1084.36 Safari/536.5",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 5.1) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_8_0) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1062.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1062.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.0 Safari/536.3",
        "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/535.24 (KHTML, like Gecko) Chrome/19.0.1055.1 Safari/535.24",
        "Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/535.24 (KHTML, like Gecko) Chrome/19.0.1055.1 Safari/535.24"
    ]

    #Ruta XPATH hacia los enlaces que queremos recopilar
    xpath_href=XPATH_HREF

    #variable que permite listar todos los enlaces recopilado
    url_list = []
    
    def _init_(self, URL_SEED, XPATH_HREF):        
        pass

    
    def get(self,url):
        session = HTMLSession()
        header = dict()
        header['user-agent'] = random.choice(self.USER_AGENT_LIST)
        response = session.get(url,headers=header)        
        return response
    
    def parse(self, response):
        self.url_list = []
        urls = response.html.xpath(self.xpath_href)
        for url in urls:
            self.url_list.append(url)


In [4]:
urls=[]

for page in tqdm(range(3)): # revisemos sólo 2 páginas
    crawler = Crawler()
    response = crawler.get('https://eltrabajo.cl/portal/author/diario-el-trabajo/page/'+str(page))
    crawler.parse(response)

    
    for url in crawler.url_list:
        urls.append(url)

    crawler.url_list = urls
    
crawler.url_list

  0%|          | 0/3 [00:00<?, ?it/s]

['https://eltrabajo.cl/portal/cerca-de-70-millones-robaron-delincuentes-de-sucursal-de-kovacs-san-felipe/',
 'https://eltrabajo.cl/portal/compleja-situacion-vive-grupo-de-extrabajadores-del-club-centro-arabe/',
 'https://eltrabajo.cl/portal/con-heridas-graves-resulta-mujer-que-fue-apunalada-en-villa-departamental/',
 'https://eltrabajo.cl/portal/a-pulso-roban-caja-fuerte-del-cementerio-parque-almendral-en-la-troya/',
 'https://eltrabajo.cl/portal/delegada-presidencial-provincial-scarlett-valdes-a-solas-con-diario-el-trabajo/',
 'https://eltrabajo.cl/portal/alimentacion-saludable-y-muchos-juegos-en-el-dia-mundial-del-deporte/',
 'https://eltrabajo.cl/portal/toda-una-semana-de-alegria-vivieron-escolares-en-el-dia-mundial-del-deporte/',
 'https://eltrabajo.cl/portal/vecinos-de-michimalonco-protestan-por-incumplimiento-de-medidas-de-mitigacion/',
 'https://eltrabajo.cl/portal/abren-campana-a-beneficio-para-que-florencia-tapia-pueda-correr-en-paralimpicos/',
 'https://eltrabajo.cl/portal/re

In [5]:
len(urls)

30

In [6]:
urls[0]

'https://eltrabajo.cl/portal/cerca-de-70-millones-robaron-delincuentes-de-sucursal-de-kovacs-san-felipe/'

In [7]:
URL_NEWS = urls[0]
XPATH_TITLE = "//header[@class='td-post-title']/h1"
XPATH_CONTENT ="//div[@class='td-post-content tagdiv-type']//p"

In [8]:
class Scraper:
    
    USER_AGENT_LIST = [
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/22.0.1207.1 Safari/537.1",
        "Mozilla/5.0 (X11; CrOS i686 2268.111.0) AppleWebKit/536.11 (KHTML, like Gecko) Chrome/20.0.1132.57 Safari/536.11",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.6 (KHTML, like Gecko) Chrome/20.0.1092.0 Safari/536.6",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.6 (KHTML, like Gecko) Chrome/20.0.1090.0 Safari/536.6",
        "Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/19.77.34.5 Safari/537.1",
        "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/536.5 (KHTML, like Gecko) Chrome/19.0.1084.9 Safari/536.5",
        "Mozilla/5.0 (Windows NT 6.0) AppleWebKit/536.5 (KHTML, like Gecko) Chrome/19.0.1084.36 Safari/536.5",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 5.1) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_8_0) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1062.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1062.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.0 Safari/536.3",
        "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/535.24 (KHTML, like Gecko) Chrome/19.0.1055.1 Safari/535.24",
        "Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/535.24 (KHTML, like Gecko) Chrome/19.0.1055.1 Safari/535.24"
    ]

    query_extract_title = XPATH_TITLE
    query_extract_text =  XPATH_CONTENT

    def _init_(self):
        pass
   
    def get(self,url):
        headers = {'user-agent':random.choice(self.USER_AGENT_LIST) }
        session = HTMLSession()
        response = session.get(url,headers=headers) 
        return response
    
    def clean_text(self, html_text):
        text_without_tags = w3lib.html.remove_tags(html_text)
        text_without_escape_chars = w3lib.html.replace_escape_chars(text_without_tags)
        text_without_escape_chars = html.unescape(text_without_escape_chars)
        text_without_whitespace = text_without_escape_chars.strip()
        return text_without_whitespace

    
    #### main function ####
    def parse(self, response):
        
        #URL
        self.news_url = response.url
        
        #TITLE
        self.news_title = response.html.xpath(self.query_extract_title)[0].text
        self.news_title = self.clean_text(self.news_title)
       
        
        #CONTENT
        self.news_text = ""
        
        html_paragraphs = response.html.xpath(self.query_extract_text)
        
        for paragraph in html_paragraphs:
            paragraph_content = w3lib.html.remove_tags(paragraph.text)    
            text_cleaned = self.clean_text(paragraph_content)
            self.news_text = self.news_text+text_cleaned


In [9]:
scraper = Scraper()
response =scraper.get(URL_NEWS)

In [10]:
scraper.parse(response)

In [11]:
scraper.news_title

'Cerca de $70 millones robaron delincuentes de sucursal de Kovacs San Felipe'

In [12]:
scraper.news_text

'Un violento robo sufrió la sucursal de Kovacs de San Felipe, ubicada en la Avenida O´Higgins, y es que en horas de la madrugada de ayer, un grupo de seis delincuentes irrumpieron de manera violenta al recinto y se dirigieron directamente a la caja fuerte del establecimiento.El gerente de Kovacs, Giovanni Depaoli, señaló que «la verdad es que yo creo que hay más personas que participaron, pero los que ingresaron al local fueron seis, los seis armados, con pistolas y con mucha violencia por así decirlo, golpearon al guardia, lo encerraron en un comedor, le quitaron las llaves, él estaba haciendo su ronda de chequeo, le quitaron además todas sus pertenencias».Fue el mismo gerente quien detalló además el momento de angustia y miedo que vivió el guardia de seguridad. Depaoli sostuvo que «él lo pasó muy mal, es lo que más nos preocupa, lo que más nos importa es la vida de las personas, él lo pasó bastante mal, lo amenazaron de muerte, estuvieron a punto de matarlo, permanentemente le ponían

In [13]:
#Verificamos que el scraper devuelve algo
assert(len(scraper.news_title)>0)
assert(len(scraper.news_text)>0)

In [14]:
df = pd.DataFrame(data={'url': [URL_NEWS],'title':[scraper.news_title], 'content':[scraper.news_text]})
df

,url,title,content
0,https://eltrabajo.cl/portal/cerca-de-70-millon...,Cerca de $70 millones robaron delincuentes de ...,Un violento robo sufrió la sucursal de Kovacs ...


In [15]:
XPATH_TITLE = "//header[@class='td-post-title']/h1"
XPATH_CONTENT ="//div[@class='td-post-content tagdiv-type']//p"

for url in tqdm(urls):
    URL_NEWS = url
    scraper = Scraper()
    response =scraper.get(URL_NEWS)
    scraper.parse(response)
    df1 = pd.DataFrame(data={'url': url, 'title':scraper.news_title, 'content':scraper.news_text},index=['url', 'title', 'content'])
    df = df.append(df1, ignore_index=True)
df = df.drop_duplicates()


  0%|          | 0/30 [00:00<?, ?it/s]

In [16]:
df

,url,title,content
0,https://eltrabajo.cl/portal/cerca-de-70-millon...,Cerca de $70 millones robaron delincuentes de ...,Un violento robo sufrió la sucursal de Kovacs ...
4,https://eltrabajo.cl/portal/compleja-situacion...,Compleja situación vive grupo de extrabajadore...,Una situación bastante compleja y hasta incomp...
7,https://eltrabajo.cl/portal/con-heridas-graves...,Con heridas graves resulta mujer que fue apuña...,Una mujer de 47 años de edad resultó con una h...
10,https://eltrabajo.cl/portal/a-pulso-roban-caja...,A pulso roban caja fuerte del Cementerio Parqu...,Tardaron menos de 10 minutos:Una banda integra...
13,https://eltrabajo.cl/portal/delegada-presidenc...,Delegada presidencial provincial Scarlett Vald...,Nos explica por qué el Presidente Boric la ins...
16,https://eltrabajo.cl/portal/alimentacion-salud...,Alimentación saludable y muchos juegos en el D...,"Peques de la Escuela San Rafael, profes y apod..."
19,https://eltrabajo.cl/portal/toda-una-semana-de...,Toda una semana de alegría vivieron escolares ...,"Peques recorrieron patios, calles y cerros de ..."
22,https://eltrabajo.cl/portal/vecinos-de-michima...,Vecinos de Michimalonco protestan por incumpli...,Encendieron barricadas:Durante la noche del pa...
25,https://eltrabajo.cl/portal/abren-campana-a-be...,Abren campaña a beneficio para que Florencia T...,"Se necesitan premios, dinero y mucho apoyo par..."
28,https://eltrabajo.cl/portal/realizan-campana-p...,Realizan campaña para ayudar a conocida comerc...,Hospitalizada en Vallenar:Una campaña destinad...
